In [1]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import twitter_samples, stopwords
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk import FreqDist, classify, NaiveBayesClassifier

In [2]:
import re, string, random, nltk

In [3]:
def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

In [4]:
def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

In [5]:
def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

In [6]:
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')
neutral_tweets = twitter_samples.strings('tweets.20150430-223406.json')

In [7]:
stop_words = stopwords.words('english')

In [8]:
positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')
neutral_tweet_tokens = twitter_samples.tokenized('tweets.20150430-223406.json')

In [9]:
positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []
neutral_cleaned_tokens_list = []

In [10]:
for tokens in positive_tweet_tokens:
        positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

for tokens in negative_tweet_tokens:
        negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))
        
for tokens in neutral_tweet_tokens:
        neutral_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

In [11]:
positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)
neutral_tokens_for_model = get_tweets_for_model(neutral_cleaned_tokens_list)

In [12]:
positive_dataset = [(tweet_dict, "Positive")
                         for tweet_dict in positive_tokens_for_model]

negative_dataset = [(tweet_dict, "Negative")
                         for tweet_dict in negative_tokens_for_model]

neutral_dataset = [(tweet_dict, "Neutral")
                         for tweet_dict in neutral_tokens_for_model]

In [13]:
dataset = positive_dataset + negative_dataset + neutral_dataset

In [14]:
 random.shuffle(dataset)

In [15]:
print(len(dataset))

30000


In [16]:
print(len(positive_dataset))
print(len(negative_dataset))
print(len(neutral_dataset))

5000
5000
20000


In [17]:
train_data = dataset[:25000]
test_data = dataset[5000:]

In [18]:
classifier = nltk.NaiveBayesClassifier.train(train_data)

In [19]:
print("Accuracy is:", classify.accuracy(classifier, test_data))

Accuracy is: 0.996


In [20]:
print(classifier.show_most_informative_features(10))

Most Informative Features
                      :( = True           Negati : Neutra =   4213.2 : 1.0
                      :d = True           Positi : Neutra =   1401.7 : 1.0
                      :) = True           Positi : Negati =   1174.4 : 1.0
                     :-( = True           Negati : Neutra =    643.6 : 1.0
                      rt = True           Neutra : Negati =    502.9 : 1.0
                      :p = True           Positi : Neutra =    321.2 : 1.0
                     :-) = True           Positi : Neutra =    268.2 : 1.0
                    tory = True           Neutra : Positi =    251.7 : 1.0
                   david = True           Neutra : Negati =    227.5 : 1.0
                 morning = True           Positi : Neutra =    211.0 : 1.0
None


In [21]:
custom_tweet = "i'm sad"

custom_tokens = remove_noise(word_tokenize(custom_tweet))

print(classifier.classify(dict([token, True] for token in custom_tokens)))

Negative


In [22]:
import pickle

In [25]:
path = "../model/nltk.NaiveBayesClassifier.pkl"
file = "nltk.NaiveBayesClassifier.pkl"

with open(path, 'wb') as file:  
    pickle.dump(classifier, file)

In [26]:
# Load the Model back from file
with open(path, 'rb') as file:  
    classifier = pickle.load(file)

classifier

In [27]:
print("Accuracy is:", classify.accuracy(classifier, test_data))

Accuracy is: 0.996


In [28]:
custom_tweet = "Jackson had never been so happy, and at ease, and she loved Connor with all her heart, yet she was a little sad, feeling they were growing apart."

custom_tokens = remove_noise(word_tokenize(custom_tweet))

print(classifier.classify(dict([token, True] for token in custom_tokens)))

Negative
